# Quadratic-Penalty Steepest-Descent Sudoku Solver (Jupyter Version)

This notebook reworks the original `sudokusolver.py` script【turn3file0】 into a **continuous optimization** Sudoku solver
using the 729-variable formulation discussed in our optimization course:

- Variables: $x_{ijk} \in \mathbb{R}_{\ge 0}$ for $i,j,k \in \{1,\dots,9\}$.
- Constraints: nonnegativity, clues, and the standard Sudoku row/column/box equalities.
- Objective: minimize
  $$
  f(x) = \sum_{i=1}^9 \sum_{j=1}^9 \sum_{k=1}^9 x_{ijk},
  $$
  subject to those constraints, approximated by a **quadratic penalty method**:
  $$
  P_2(x,\rho) = f(x) + \frac{\rho}{2}\,\lVert G(x) \rVert_2^2,
  $$
  where $G(x)=0$ collects all Sudoku equalities.

We solve $\min_x P_2(x,\rho)$ for an increasing sequence of penalty parameters $\rho$, using
**steepest descent with backtracking line search**, and then decode the resulting $x$ to an integer
Sudoku grid.

This notebook:

1. Parses `sudoku.txt` (80 puzzles at 4 difficulty levels) into NumPy arrays.
2. Implements the quadratic-penalty steepest-descent solver.
3. Solves all puzzles and reports the success rate.


In [ ]:
import numpy as np
import re
from pathlib import Path

## 1. Parsing `sudoku.txt`

The file `sudoku.txt` is an R-style dump of 80 Sudoku puzzles at four difficulty levels.
Structure (schematically):

- Top-level markers: `[[1]]`, `[[2]]`, `[[3]]`, `[[4]]` for difficulty levels.
- Per-puzzle headers: `[[d]][[p]]` for difficulty `d` and puzzle index `p` (1 to 20).
- Then an R matrix header line: `[,1] [,2] ... [,9]`.
- Then 9 row lines of the form:
  ```
   [1,]    2    4    8    9    7    3    1    6    5
  ```
  where the first token encodes the row index and is ignored; the remaining 9 tokens are integers
  in `{0,...,9}`, with `0` meaning an empty cell.

We parse this into a dictionary:
```python
puzzles_by_difficulty = {1: [grid1, ..., grid20],
                         2: [...],
                         3: [...],
                         4: [...]}
```
where each `grid` is a `9×9` NumPy array of integers.


In [ ]:
def parse_sudoku_txt(path):
    """Parse sudoku.txt into a dict {difficulty: [9x9 numpy arrays]}.

    The file is an R-like dump with entries like
    [[1]]
    [[1]][[1]]
      [,1] [,2] ... [,9]
     [1,]  ... 9 ints ...
    """
    text = Path(path).read_text().splitlines()

    puzzles_by_diff = {1: [], 2: [], 3: [], 4: []}
    current_diff = None
    current_grid = None
    current_rows = []

    # regex for headers of the form [[d]][[p]]
    header_re = re.compile(r"^\[\[(\d+)\]\]\[\[(\d+)\]\]$")

    for line in text:
        s = line.strip()
        if not s:
            continue

        # Detect per-puzzle header [[d]][[p]]
        m = header_re.match(s)
        if m:
            diff = int(m.group(1))
            # Start a new puzzle
            current_diff = diff
            current_grid = []
            current_rows = []
            continue

        # Skip matrix column header lines like "[,1] [,2] ..."
        if s.startswith("[,1]"):
            continue

        # Row lines start with something like "[1,]" then 9 integers
        if s.startswith("[") or s.startswith("[1,") or s.startswith("[2,") or s.startswith("[3,"):
            # Generic: first token is like "[1,]"; ignore it
            tokens = s.split()
            if tokens[0].startswith("["):
                tokens = tokens[1:]
            # Now tokens should be 9 integers
            if len(tokens) >= 9:
                row_vals = [int(t) for t in tokens[:9]]
                current_rows.append(row_vals)

                # When we have 9 rows, store the grid
                if len(current_rows) == 9:
                    grid = np.array(current_rows, dtype=int)
                    if current_diff is None:
                        raise ValueError("Found grid rows before difficulty header.")
                    puzzles_by_diff[current_diff].append(grid)
                    current_rows = []
            continue

    # Basic sanity checks
    for d in puzzles_by_diff:
        if len(puzzles_by_diff[d]) == 0:
            print(f"Warning: difficulty {d} has no puzzles parsed.")
    return puzzles_by_diff


# Parse sudoku.txt (assumed in the same directory as this notebook)
SUDOKU_PATH = "sudoku.txt"

puzzles_by_diff = parse_sudoku_txt(SUDOKU_PATH)
for d in sorted(puzzles_by_diff):
    print(f"Difficulty {d}: {len(puzzles_by_diff[d])} puzzles parsed.")

# Show the first puzzle of difficulty 1
if puzzles_by_diff[1]:
    print("\nDifficulty 1, puzzle 1 (0 = empty):")
    print(puzzles_by_diff[1][0])

## 2. Continuous 729-variable formulation and constraint builder

We flatten indices $(i,j,k)$ with $i,j,k \in \{0,\dots,8\}$ (0-based) into a single index
$p \in \{0,\dots,728\}$ via
$$
p = i\cdot 81 + j\cdot 9 + k.
$$

We then build a list of linear equality constraints of the form
$$
g_\ell(x) = \sum_{p \in S_\ell} x_p - c_\ell = 0,
$$
for:

- **Cell constraints**: each cell $(i,j)$ has exactly one digit,
  $$\sum_k x_{ijk} = 1.$$
- **Row-digit constraints**: each digit $k$ appears exactly once in row $i$,
  $$\sum_j x_{ijk} = 1.$$
- **Column-digit constraints**: each digit $k$ appears exactly once in column $j$,
  $$\sum_i x_{ijk} = 1.$$
- **Box-digit constraints**: each digit $k$ appears exactly once in each 3×3 box.
- **Clue constraints**: if the puzzle has $a_{ij} = k+1$, then $x_{ijk} = 1$.

We store each constraint as a pair `(indices, c)` where `indices` is a list of flattened indices
and `c` is the right-hand side constant (usually 1; for clues, 1 as well).


In [ ]:
def idx(i, j, k):
    """Flatten (i,j,k) with i,j,k ∈ {0,...,8} into p ∈ {0,...,728}."""
    return i * 81 + j * 9 + k


def build_constraints_for_grid(grid):
    """Build Sudoku equality constraints for a given 9x9 integer grid.

    grid[i,j] ∈ {0,...,9}, with 0 = empty.
    Returns a list of (indices, c) constraints.
    """
    constraints = []

    # Cell constraints: sum_k x_{ijk} = 1
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            constraints.append((indices, 1.0))

    # Row-digit constraints: sum_j x_{ijk} = 1
    for i in range(9):
        for k in range(9):
            indices = [idx(i, j, k) for j in range(9)]
            constraints.append((indices, 1.0))

    # Column-digit constraints: sum_i x_{ijk} = 1
    for j in range(9):
        for k in range(9):
            indices = [idx(i, j, k) for i in range(9)]
            constraints.append((indices, 1.0))

    # Box-digit constraints: sum_{(i,j) in box} x_{ijk} = 1
    for box_row in (0, 3, 6):
        for box_col in (0, 3, 6):
            for k in range(9):
                indices = []
                for i in range(box_row, box_row + 3):
                    for j in range(box_col, box_col + 3):
                        indices.append(idx(i, j, k))
                constraints.append((indices, 1.0))

    # Clue constraints: x_{ijk} = 1 when grid[i,j] = k+1
    for i in range(9):
        for j in range(9):
            val = grid[i, j]
            if val != 0:
                k = val - 1  # digit k+1
                indices = [idx(i, j, k)]
                constraints.append((indices, 1.0))

    return constraints


# Quick check on first puzzle
if puzzles_by_diff[1]:
    constraints_example = build_constraints_for_grid(puzzles_by_diff[1][0])
    print(f"Number of constraints for one puzzle: {len(constraints_example)}")

## 3. Quadratic penalty function, gradient, and projection

We define the **penalized objective**

$$
P_2(x, \rho) = f(x) + \frac{\rho}{2} \sum_\ell g_\ell(x)^2,
$$

where

- $f(x) = \sum_p x_p$ is the original objective (constant on the exact feasible set),
- each constraint is $g_\ell(x) = \sum_{p\in S_\ell} x_p - c_\ell$,
- $\rho > 0$ is the quadratic penalty parameter.

The gradient is

$$
\nabla P_2(x, \rho)
= \mathbf{1} + \rho \sum_\ell g_\ell(x)\, \nabla g_\ell(x),
$$

and since each $g_\ell$ is linear with coefficients 1 on its support $S_\ell$, this becomes

$$
[\nabla P_2(x, \rho)]_p
= 1 + \rho \sum_{\ell:\, p\in S_\ell} g_\ell(x).
$$

We implement this matrix-free, storing each constraint as `(indices, c)`.
We also enforce nonnegativity via projection:
$$
x \leftarrow \max(x, 0) \quad \text{(componentwise)}.
$$


In [ ]:
def P2(x, rho, constraints, mu=0.0):
    """Quadratic penalty objective P2(x, rho) with optional integrality term.
    
    The objective includes:
    - f(x) = sum(x): sparsity-encouraging linear term
    - (rho/2) * ||G(x)||^2: quadratic penalty for constraint violations
    - mu * sum(x*(1-x)): integrality-encouraging term (pushes x toward 0 or 1)
    """
    f = x.sum()
    penalty = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        penalty += r * r
    # Integrality-encouraging term: x*(1-x) is maximized at x=0.5, zero at x=0 or x=1
    integrality_penalty = np.sum(x * (1.0 - x))
    return f + 0.5 * rho * penalty + mu * integrality_penalty


def grad_P2(x, rho, constraints, mu=0.0):
    """Gradient of P2(x, rho, mu) (matrix-free).
    
    Gradient includes:
    - 1 (from f(x) = sum(x))
    - rho * sum of constraint residuals for constraints containing x_p
    - mu * (1 - 2*x) (from x*(1-x) term)
    """
    g = np.ones_like(x)  # gradient of f(x) = sum x
    for indices, c in constraints:
        r = x[indices].sum() - c
        if r != 0.0:
            g[indices] += rho * r
    # Add gradient of integrality term: d/dx [x*(1-x)] = 1 - 2*x
    g += mu * (1.0 - 2.0 * x)
    return g


def constraint_violation_norm(x, constraints):
    """Return ||G(x)||_2, where G collects all g_ℓ(x)."""
    sumsq = 0.0
    for indices, c in constraints:
        r = x[indices].sum() - c
        sumsq += r * r
    return np.sqrt(sumsq)


def project_nonnegative(x):
    """Project x onto the nonnegative orthant (in-place)."""
    np.maximum(x, 0.0, out=x)
    return x


def encode_grid_to_x(grid):
    """Encode a 9x9 integer Sudoku grid into a 729-dimensional binary vector.
    
    For a valid complete grid, returns the exact binary indicator x.
    For a partial grid (with zeros), sets x_ijk = 1/9 for empty cells.
    This is useful for diagnostics and initializing from a known solution.
    """
    x = np.zeros(9 * 9 * 9)
    for i in range(9):
        for j in range(9):
            val = grid[i, j]
            if val != 0:
                # Known value: set x_{ij,val-1} = 1
                x[idx(i, j, val - 1)] = 1.0
            else:
                # Empty cell: uniform distribution over digits
                for k in range(9):
                    x[idx(i, j, k)] = 1.0 / 9.0
    return x


def diagnose_constraint_violations(x, constraints, verbose=True):
    """Print diagnostics about constraint violations."""
    violations = []
    for i, (indices, c) in enumerate(constraints):
        r = x[indices].sum() - c
        if abs(r) > 1e-4:
            violations.append((i, r, indices))
    
    if violations and verbose:
        print(f"  Found {len(violations)} constraint violations (|r| > 1e-4):")
        for i, r, indices in violations[:5]:  # Show first 5
            print(f"    Constraint {i}: residual={r:.4f}")
        if len(violations) > 5:
            print(f"    ... and {len(violations) - 5} more")
    elif verbose:
        print("  All constraints satisfied within tolerance.")
    return len(violations)

## 4. Steepest descent with quadratic penalty and Sudoku decoding

For a fixed penalty parameter $\rho$, we approximately minimize $P_2(x, \rho)$ with **steepest descent**:

$$
x^{t+1} = \Pi_{x\ge 0}\bigl(x^t - \alpha_t \nabla P_2(x^t, \rho)\bigr),
$$

with backtracking line search (Armijo condition) to pick $\alpha_t$.

We then use an **outer loop** over increasing $\rho$ values, reusing the minimizer from one
penalty parameter as the starting point for the next.

Finally, we decode the continuous $x$ into a Sudoku grid by, for each cell $(i,j)$, selecting
the digit $k$ with the largest $x_{ijk}$ (argmax). We then validate the resulting integer grid.


In [ ]:
def steepest_descent_penalty(x0, rho, constraints,
                             mu=0.0,
                             max_iter_inner=5000,
                             tol_grad=1e-6,
                             tol_constr=1e-6,
                             alpha_init=1.0,
                             beta=0.5):
    """Inner steepest-descent loop for a fixed penalty parameter rho.
    
    Parameters:
    - x0: initial point
    - rho: penalty parameter for constraint violations
    - constraints: list of (indices, c) constraint tuples
    - mu: integrality penalty weight (default 0)
    - max_iter_inner: maximum iterations
    - tol_grad: gradient norm tolerance
    - tol_constr: constraint violation tolerance
    - alpha_init: initial step size for backtracking
    - beta: backtracking factor
    """
    x = x0.copy()
    for t in range(max_iter_inner):
        g = grad_P2(x, rho, constraints, mu)
        grad_norm = np.linalg.norm(g)
        if grad_norm < tol_grad:
            break

        P_current = P2(x, rho, constraints, mu)
        alpha = alpha_init
        c_armijo = 1e-4

        # Backtracking line search
        while True:
            x_trial = project_nonnegative(x - alpha * g)
            P_trial = P2(x_trial, rho, constraints, mu)
            if P_trial <= P_current - c_armijo * alpha * grad_norm**2:
                break
            alpha *= beta
            if alpha < 1e-10:
                x_trial = x
                break

        x = x_trial
        constr_norm = constraint_violation_norm(x, constraints)
        if constr_norm < tol_constr:
            break

    return x


def is_near_integral(x, tol_peak=0.9, tol_rest=0.15):
    """Heuristic: check if x is close to 0/1 binary.
    
    For each cell (i,j), checks that one digit k has x_{ijk} > tol_peak
    and all others have x_{ijk} < tol_rest.
    """
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            cell_vals = x[indices]
            k_star = np.argmax(cell_vals)
            if cell_vals[k_star] < tol_peak:
                return False
            if np.any(np.delete(cell_vals, k_star) > tol_rest):
                return False
    return True


def decode_solution(x):
    """Decode continuous x into a 9x9 integer Sudoku grid via argmax over k."""
    grid = np.zeros((9, 9), dtype=int)
    for i in range(9):
        for j in range(9):
            indices = [idx(i, j, k) for k in range(9)]
            k_star = int(np.argmax(x[indices]))
            grid[i, j] = k_star + 1
    return grid


def is_valid_sudoku(grid, original_grid):
    """Check whether `grid` is a valid Sudoku solution respecting `original_grid` clues."""
    # Clues respected
    for i in range(9):
        for j in range(9):
            if original_grid[i, j] != 0 and grid[i, j] != original_grid[i, j]:
                return False

    # Rows
    target = set(range(1, 10))
    for i in range(9):
        if set(grid[i, :]) != target:
            return False

    # Columns
    for j in range(9):
        if set(grid[:, j]) != target:
            return False

    # Boxes
    for bi in (0, 3, 6):
        for bj in (0, 3, 6):
            block = grid[bi:bi+3, bj:bj+3].ravel()
            if set(block) != target:
                return False

    return True


# Default integrality weight scale factor (mu_scale * rho/rho_max in later stages)
DEFAULT_MU_SCALE = 0.1


def solve_sudoku_continuous(grid,
                            rho_list=(1.0, 10.0, 100.0, 1000.0, 5000.0, 20000.0),
                            mu_list=None,
                            mu_scale=DEFAULT_MU_SCALE,
                            max_iter_inner=3000,
                            verbose=False):
    """Solve a single Sudoku puzzle via quadratic-penalty steepest descent.

    Parameters:
    - grid: 9x9 numpy array with 0 for empty cells
    - rho_list: increasing sequence of penalty parameters
    - mu_list: optional sequence of integrality weights (same length as rho_list)
    - mu_scale: scale factor for auto-generated mu schedule (default 0.1)
    - max_iter_inner: max iterations per penalty stage
    - verbose: print progress information

    Returns (solution_grid, history), where history is a list of
    (rho, mu, constr_norm, P2_value) tuples.
    """
    constraints = build_constraints_for_grid(grid)

    # Initialize x using clues and uniform distribution for empty cells
    x = encode_grid_to_x(grid)

    # Default mu schedule: starts at 0 to let penalty dominate initially,
    # then gradually increases to encourage integrality in later stages.
    # The integrality term mu*x*(1-x) helps push x toward 0/1 values.
    if mu_list is None:
        mu_list = [0.0] * len(rho_list)
        # Enable integrality term only in the second half of the schedule
        # to avoid interfering with initial constraint satisfaction
        for i in range(len(rho_list) // 2, len(rho_list)):
            mu_list[i] = mu_scale * rho_list[i] / rho_list[-1]

    history = []

    for rho, mu in zip(rho_list, mu_list):
        x = steepest_descent_penalty(
            x0=x,
            rho=rho,
            constraints=constraints,
            mu=mu,
            max_iter_inner=max_iter_inner,
            tol_grad=1e-7,
            tol_constr=1e-7,
            alpha_init=1.0,
            beta=0.6,
        )
        constr_norm = constraint_violation_norm(x, constraints)
        P_val = P2(x, rho, constraints, mu)
        history.append((rho, mu, constr_norm, P_val))
        
        if verbose:
            print(f"  rho={rho:g}, mu={mu:.3f}, ||G||={constr_norm:.2e}, near_int={is_near_integral(x)}")
        
        # Early exit if constraints satisfied and near-integral
        if constr_norm < 1e-5 and is_near_integral(x):
            break

    solution_grid = decode_solution(x)
    return solution_grid, history

## 5. Solving all puzzles from `sudoku.txt`

We now run the continuous quadratic-penalty solver on each of the 80 puzzles and report, per
difficulty level:

- Whether the solution is valid.
- Final constraint violation norm and penalized objective value.
- A small snippet of the solution for inspection.

This is primarily pedagogical: it illustrates how the quadratic penalty method behaves on a
discrete/combinatorial problem, rather than being a production-grade Sudoku solver.


In [ ]:
# Summarize shape info for `puzzles_by_diff` (works if it's a dict of grids)
if isinstance(puzzles_by_diff, dict):
    for d in sorted(puzzles_by_diff):
        grids = puzzles_by_diff[d]
        shapes = {getattr(g, "shape", None) for g in grids}
        print(f"Difficulty {d}: {len(grids)} puzzles; grid shapes: {shapes}")
else:
    try:
        print("shape:", puzzles_by_diff.shape)
    except Exception as e:
        print("No .shape attribute; type:", type(puzzles_by_diff), "repr (truncated):", repr(puzzles_by_diff)[:200])

In [ ]:
def solve_all_puzzles(puzzles_by_diff,
                        rho_list=(1.0, 10.0, 100.0, 1000.0, 5000.0, 20000.0),
                        max_iter_inner=3000,
                        verbose=False):
    """Solve all puzzles in the dataset.
    
    Parameters:
    - puzzles_by_diff: dict {difficulty: [grids]}
    - rho_list: penalty schedule
    - max_iter_inner: iterations per penalty stage
    - verbose: show per-iteration progress
    """
    results = {}
    for d in sorted(puzzles_by_diff):
        puzzles = puzzles_by_diff[d]
        print(f"\n=== Difficulty {d} ===")
        success_count = 0
        total = len(puzzles)
        for idx_p, grid in enumerate(puzzles, start=21):
            print(f"Puzzle {idx_p}:")
            solution, history = solve_sudoku_continuous(
                grid,
                rho_list=rho_list,
                max_iter_inner=max_iter_inner,
                verbose=verbose,
            )
            valid = is_valid_sudoku(solution, grid)
            if valid:
                success_count += 1
            final_rho, final_mu, final_constr, final_P2 = history[-1]
            print(f"  valid: {valid}; final constraint norm {final_constr:.2e}")
            print("  history (rho, mu, constr_norm, P2):")
            for rho, mu, cn, p2 in history:
                print(f"    rho={rho:g}, mu={mu:.3f}, constr_norm={cn:.2e}, P2={p2:.3f}")
            # Show top-left 3x3 block as a quick check
            print("  top-left 3x3 block:")
            print(solution[:3, :3])
        results[d] = (success_count, total)
        rate = 100.0 * success_count / max(total, 1)
        print(f"\nSolved {success_count} / {total} puzzles for difficulty {d} (success rate {rate:.1f}%).")
    return results


def diagnose_sudoku_failures(solution, original_grid):
    """Print details about why a Sudoku solution is invalid."""
    target = set(range(1, 10))
    issues = []
    
    # Check clues respected
    for i in range(9):
        for j in range(9):
            if original_grid[i, j] != 0 and solution[i, j] != original_grid[i, j]:
                issues.append(f"Clue violated at ({i},{j}): expected {original_grid[i,j]}, got {solution[i,j]}")
    
    # Check rows
    for i in range(9):
        row_set = set(solution[i, :])
        if row_set != target:
            missing = target - row_set
            duplicate = [v for v in solution[i, :] if list(solution[i, :]).count(v) > 1]
            issues.append(f"Row {i}: missing {missing}, duplicates {set(duplicate)}")
    
    # Check columns
    for j in range(9):
        col_set = set(solution[:, j])
        if col_set != target:
            missing = target - col_set
            issues.append(f"Column {j}: missing {missing}")
    
    # Check boxes
    for bi in (0, 3, 6):
        for bj in (0, 3, 6):
            block = solution[bi:bi+3, bj:bj+3].ravel()
            block_set = set(block)
            if block_set != target:
                missing = target - block_set
                issues.append(f"Box ({bi//3},{bj//3}): missing {missing}")
    
    if issues:
        print("  Issues found:")
        for issue in issues[:5]:  # Show first 5 issues
            print(f"    - {issue}")
        if len(issues) > 5:
            print(f"    ... and {len(issues) - 5} more")
    return len(issues)


def quick_test(puzzles_by_diff, difficulties=(1, 2, 3), puzzles_per_diff=2, verbose=True):
    """Run quick tests on a limited number of puzzles for debugging.
    
    Parameters:
    - puzzles_by_diff: dict {difficulty: [grids]}
    - difficulties: which difficulty levels to test
    - puzzles_per_diff: how many puzzles per difficulty
    - verbose: show detailed progress
    
    Returns dict of results.
    """
    print("=== Quick Test Mode ====")
    print(f"Testing {puzzles_per_diff} puzzles per difficulty level {difficulties}\n")
    
    results = {}
    for d in difficulties:
        if d not in puzzles_by_diff:
            continue
        print(f"\n--- Difficulty {d} ---")
        puzzles = puzzles_by_diff[d][:puzzles_per_diff]
        success_count = 0
        
        for idx_p, grid in enumerate(puzzles, start=1):
            print(f"\nPuzzle {idx_p}:")
            solution, history = solve_sudoku_continuous(
                grid,
                verbose=verbose,
            )
            valid = is_valid_sudoku(solution, grid)
            if valid:
                success_count += 1
                print(f"  PASSED - Valid solution found")
            else:
                print(f"  FAILED - Invalid solution")
                # Show detailed diagnostics for failed puzzles
                diagnose_sudoku_failures(solution, grid)
                print("  Solution grid:")
                print(solution)
        
        results[d] = (success_count, len(puzzles))
        print(f"\nDifficulty {d}: {success_count}/{len(puzzles)} passed")
    
    total_passed = sum(r[0] for r in results.values())
    total_tested = sum(r[1] for r in results.values())
    print(f"\n=== Overall: {total_passed}/{total_tested} puzzles passed ===")
    return results


# Run quick test on limited puzzles (faster for debugging)
quick_results = quick_test(puzzles_by_diff, difficulties=(1, 2), puzzles_per_diff=3)

## Part e: `checkSudoku`

In [ ]:
import numpy as np

def x_to_grid(x):
    """
    Convert a 729-dimensional solution vector x_* ∈ R^{729}
    into a conventional 9×9 integer Sudoku solution A_*.

    This is just a thin wrapper around `decode_solution(x)`.
    """
    x = np.asarray(x, dtype=float).ravel()
    if x.shape[0] != 9 * 9 * 9:
        raise ValueError(f"x must have length 729, got {x.shape[0]}")
    return decode_solution(x)


def checkSudoku(A, original_grid=None, raise_on_error=True, verbose=True):
    """
    Check that A_* ∈ Z^{9×9} is a valid Sudoku solution:
      - entries in {1,…,9}
      - each row contains 1,…,9 exactly once
      - each column contains 1,…,9 exactly once
      - each 3×3 box contains 1,…,9 exactly once
    Optionally also check that given clues (original_grid) are respected.

    Parameters
    ----------
    A : array-like
        Candidate 9×9 integer grid.
    original_grid : array-like or None
        9×9 grid of the puzzle (0 = empty). If provided, nonzero entries
        must match A.
    raise_on_error : bool
        If True, raise a ValueError with the first detected issue.
        If False, return False on failure.
    verbose : bool
        If False, do not print messages; just raise/return.

    Returns
    -------
    bool
        True if A passes all checks; False otherwise (if raise_on_error=False).
    """
    grid = np.asarray(A, dtype=int)
    if grid.shape != (9, 9):
        msg = f"A must be 9×9, got shape {grid.shape}"
        if raise_on_error:
            raise ValueError(msg)
        if verbose:
            print("checkSudoku:", msg)
        return False

    digits = set(range(1, 10))

    # Check all entries are in {1,...,9}
    if not np.all((grid >= 1) & (grid <= 9)):
        msg = "Grid contains entries outside {1,…,9}."
        if raise_on_error:
            raise ValueError(msg)
        if verbose:
            print("checkSudoku:", msg)
        return False

    # Optionally enforce clues
    if original_grid is not None:
        orig = np.asarray(original_grid, dtype=int)
        if orig.shape != (9, 9):
            msg = f"original_grid must be 9×9, got shape {orig.shape}"
            if raise_on_error:
                raise ValueError(msg)
            if verbose:
                print("checkSudoku:", msg)
            return False
        for i in range(9):
            for j in range(9):
                if orig[i, j] != 0 and grid[i, j] != orig[i, j]:
                    msg = f"Clue mismatch at ({i+1},{j+1}): expected {orig[i,j]}, got {grid[i,j]}"
                    if raise_on_error:
                        raise ValueError(msg)
                    if verbose:
                        print("checkSudoku:", msg)
                    return False

    # Rows
    for i in range(9):
        row = grid[i, :]
        if set(row) != digits:
            msg = f"Row {i+1} invalid: {row}"
            if raise_on_error:
                raise ValueError(msg)
            if verbose:
                print("checkSudoku:", msg)
            return False

    # Columns
    for j in range(9):
        col = grid[:, j]
        if set(col) != digits:
            msg = f"Column {j+1} invalid: {col}"
            if raise_on_error:
                raise ValueError(msg)
            if verbose:
                print("checkSudoku:", msg)
            return False

    # 3×3 boxes
    for bi in (0, 3, 6):
        for bj in (0, 3, 6):
            block = grid[bi:bi+3, bj:bj+3].ravel()
            if set(block) != digits:
                msg = f"3×3 box at rows {bi+1}–{bi+3}, cols {bj+1}–{bj+3} invalid: {block}"
                if raise_on_error:
                    raise ValueError(msg)
                if verbose:
                    print("checkSudoku:", msg)
                return False

    if verbose:
        print("checkSudoku: grid is a valid Sudoku solution.")
    return True



In [ ]:
toy_grid = puzzles_by_diff[1][0]           # first puzzle of difficulty 1
solution_x, history = solve_sudoku_continuous(toy_grid, verbose=False)

# Make the conversion explicit (x_* → A_*)
A_star = x_to_grid(solution_x)

# Now checkSudoku(A_*) against the original puzzle
checkSudoku(A_star, original_grid=toy_grid, raise_on_error=True, verbose=True)
